In [1]:
%matplotlib inline

# # silence warnings
# import warnings
# warnings.filterwarnings('ignore')
import pyfolio as pf
import pandas_datareader as pdr
import pandas_datareader.data as web
import os
import quandl

In [2]:
api_key = os.environ["QUANDL_API_KEY"]
quandl.ApiConfig.api_key = api_key


In [3]:
symbol = 'FB'  # or 'AAPL.US'
df = web.DataReader(symbol, 'quandl', '2012-05-22', '2014-05-16', access_key=api_key)
df.tail()

,Open,High,Low,Close,Volume,ExDividend,SplitRatio,AdjOpen,AdjHigh,AdjLow,AdjClose,AdjVolume
Date,,,,,,,,,,,,
2012-05-29,31.48,31.69,28.65,28.84,78063400.0,0.0,1.0,31.48,31.69,28.65,28.84,78063400.0
2012-05-25,32.90,32.95,31.11,31.91,37149800.0,0.0,1.0,32.90,32.95,31.11,31.91,37149800.0
2012-05-24,32.95,33.21,31.77,33.03,50237200.0,0.0,1.0,32.95,33.21,31.77,33.03,50237200.0
2012-05-23,31.37,32.50,31.36,32.00,73600000.0,0.0,1.0,31.37,32.50,31.36,32.00,73600000.0
2012-05-22,32.61,33.59,30.94,31.00,101786600.0,0.0,1.0,32.61,33.59,30.94,31.00,101786600.0


In [4]:
from empyrical.utils import get_utc_timestamp, get_returns_cached, _1_bday_ago, data_path
from pandas_datareader import data as web
from datetime import datetime
import pandas as pd

def get_symbol_returns_from_quandl(symbol, start=None, end=None):
    """
    Wrapper for pandas.io.data.get_data_quandl().
    Retrieves prices for symbol from quandl and computes returns
    based on adjusted closing prices.

    Parameters
    ----------
    symbol : str
        Symbol name to load, e.g. 'SPY'
    start : pandas.Timestamp compatible, optional
        Start date of time period to retrieve
    end : pandas.Timestamp compatible, optional
        End date of time period to retrieve

    Returns
    -------
    pandas.DataFrame
        Returns of symbol in requested period.
    """

    px = web.get_data_quandl(symbol, start=start, end=end)
    rets = px[['AdjClose']]
    rets = rets.shift(-1)
    rets.iloc[-1]['AdjClose'] = px.tail(1)['AdjOpen']
    rets = rets.shift(1) / rets - 1
    rets = rets.dropna()
    rets.index = rets.index.to_datetime()
    rets.index = rets.index.tz_localize("UTC")
    rets.columns = [symbol]
    return rets

def returns_func(symbol, start=None, end=None):
    """
    Gets returns for a symbol.
    Queries Quandl Finance. Attempts to cache SPY.

    Parameters
    ----------
    symbol : str
        Ticker symbol, e.g. APPL.
    start : date, optional
        Earliest date to fetch data for.
        Defaults to earliest date available.
    end : date, optional
        Latest date to fetch data for.
        Defaults to latest date available.

    Returns
    -------
    pd.Series
        Daily returns for the symbol.
         - See full explanation in tears.create_full_tear_sheet (returns).
    """

    if start is None:
        start = '1/1/1970'
    if end is None:
        end = _1_bday_ago()

    start = get_utc_timestamp(start)
    end = get_utc_timestamp(end)

    if symbol == 'SPY':
        filepath = data_path('spy.csv')
        rets = get_returns_cached(filepath,
                                  get_symbol_returns_from_quandl,
                                  end,
                                  symbol='SPY',
                                  start=start,
                                  end=get_utc_timestamp(datetime.now()))
        
        try:
            rets = rets[rets.index.isin(pd.bdate_range(start, end))]
        except Exception as e:
            rets = rets[start:end]
    else:
        rets = get_symbol_returns_from_quandl(symbol, start=start, end=end)
    rets.sort_index(inplace=True)
    return rets[symbol]



In [5]:
pf.utils.register_return_func(returns_func)

In [6]:
stock_rets = pf.utils.get_symbol_rets('FB', start='2012-05-22', end='2014-05-16')

In [7]:
stock_rets

Date
2012-05-22 00:00:00+00:00   -0.049371
2012-05-23 00:00:00+00:00    0.032258
2012-05-24 00:00:00+00:00    0.032188
2012-05-25 00:00:00+00:00   -0.033909
2012-05-29 00:00:00+00:00   -0.096208
2012-05-30 00:00:00+00:00   -0.022538
2012-05-31 00:00:00+00:00    0.050018
2012-06-01 00:00:00+00:00   -0.063514
2012-06-04 00:00:00+00:00   -0.029582
2012-06-05 00:00:00+00:00   -0.038327
2012-06-06 00:00:00+00:00    0.036376
2012-06-07 00:00:00+00:00   -0.018650
2012-06-08 00:00:00+00:00    0.030027
2012-06-11 00:00:00+00:00   -0.003506
2012-06-12 00:00:00+00:00    0.014627
2012-06-13 00:00:00+00:00   -0.004745
2012-06-14 00:00:00+00:00    0.037404
2012-06-15 00:00:00+00:00    0.060940
2012-06-18 00:00:00+00:00    0.046512
2012-06-19 00:00:00+00:00    0.015950
2012-06-20 00:00:00+00:00   -0.009746
2012-06-21 00:00:00+00:00    0.007595
2012-06-22 00:00:00+00:00    0.038003
2012-06-25 00:00:00+00:00   -0.029955
2012-06-26 00:00:00+00:00    0.032439
2012-06-27 00:00:00+00:00   -0.026284
2012-06

In [19]:
[stock_rets.size, stock_rets.name, stock_rets.index, stock_rets.dtype]

[498,
 'FB',
 DatetimeIndex(['2012-05-22', '2012-05-23', '2012-05-24', '2012-05-25',
                '2012-05-29', '2012-05-30', '2012-05-31', '2012-06-01',
                '2012-06-04', '2012-06-05',
                ...
                '2014-05-02', '2014-05-05', '2014-05-06', '2014-05-07',
                '2014-05-08', '2014-05-09', '2014-05-12', '2014-05-13',
                '2014-05-14', '2014-05-15'],
               dtype='datetime64[ns, UTC]', name='Date', length=498, freq=None),
 dtype('float64')]

In [1]:
import pandas as pd
import datetime
import requests
def get_yahoo_finance_data(symbol, start=None, end=None, interval='1d'):
    end = pd.to_datetime(end) or datetime.datetime.utcnow()
    start = pd.to_datetime(start) or end - datetime.timedelta(days=24)
    url = 'https://query1.finance.yahoo.com/v8/finance/chart/{symbol}?symbol={symbol}&period1={start}&period2={end}&interval={interval}&includePrePost=true&events=div%7Csplit%7Cearn&corsDomain=finance.yahoo.com'.format(
        symbol=symbol,
        start=start.strftime('%s'),
        end=end.strftime('%s'),
        interval=interval
    )
    resp = requests.get(url)
    resp.raise_for_status()
    
    rt = resp.json()
    
    return resp.json()

def get_correct_data_format(resp_json):
    symbol = rtjson['chart']['result'][0]['meta']['symbol']
    timestamps = rtjson['chart']['result'][0]['timestamp']
    open_price = rtjson['chart']['result'][0]['indicators']['quote'][0]['open']
    close_price = rtjson['chart']['result'][0]['indicators']['quote'][0]['close']
    
    ts = pd.DataFrame({'open' : pd.Series(open_price, index=pd.to_datetime(timestamps, unit='s')),
                       'close' : pd.Series(close_price, index=pd.to_datetime(timestamps, unit='s'))})
    ts.name = symbol
    ts.index = pd.to_datetime(ts.index, unit='s')
    
    return ts
    
def get_symbol_returns_custom(ts):
    rets = ts[['close']]
    rets = rets.shift(-1)
    rets.iloc[-1]['close'] = ts.tail(1)['open']
    rets = rets.shift(1) / rets - 1
    rets = rets.dropna()
    rets.index = rets.index.to_datetime()
    rets.index = rets.index.tz_localize("UTC")
    rets.columns = [ts.name]
    rets = pd.Series(rets[ts.name])
    return rets
    



In [2]:
rtjson = get_yahoo_finance_data('SPY', start='1/1/2017', end='12/1/2017')

In [3]:
ts = get_correct_data_format(rtjson)

In [4]:
get_symbol_returns_custom(ts)

2017-01-04 14:30:00+00:00    0.000795
2017-01-05 14:30:00+00:00   -0.003565
2017-01-06 14:30:00+00:00    0.003312
2017-01-09 14:30:00+00:00    0.000000
2017-01-10 14:30:00+00:00   -0.002818
2017-01-11 14:30:00+00:00    0.002516
2017-01-12 14:30:00+00:00   -0.002290
2017-01-13 14:30:00+00:00    0.003536
2017-01-17 14:30:00+00:00   -0.002205
2017-01-18 14:30:00+00:00    0.003718
2017-01-19 14:30:00+00:00   -0.003661
2017-01-20 14:30:00+00:00    0.002609
2017-01-23 14:30:00+00:00   -0.006371
2017-01-24 14:30:00+00:00   -0.008581
2017-01-25 14:30:00+00:00    0.001047
2017-01-26 14:30:00+00:00    0.001572
2017-01-27 14:30:00+00:00    0.006240
2017-01-30 14:30:00+00:00    0.000088
2017-01-31 14:30:00+00:00   -0.000395
2017-02-01 14:30:00+00:00   -0.000659
2017-02-02 14:30:00+00:00   -0.006846
2017-02-03 14:30:00+00:00    0.001791
2017-02-06 14:30:00+00:00   -0.000044
2017-02-07 14:30:00+00:00   -0.001309
2017-02-08 14:30:00+00:00   -0.005898
2017-02-09 14:30:00+00:00   -0.003931
2017-02-10 1

In [65]:
rets = ts[['close']]

In [66]:
rets

,close
2017-01-03 14:30:00,225.240005
2017-01-04 14:30:00,226.580002
2017-01-05 14:30:00,226.399994
2017-01-06 14:30:00,227.210007
2017-01-09 14:30:00,226.460007
2017-01-10 14:30:00,226.460007
2017-01-11 14:30:00,227.100006
2017-01-12 14:30:00,226.529999
2017-01-13 14:30:00,227.050003
2017-01-17 14:30:00,226.250000


In [68]:
rets = rets.shift(-1)

In [69]:
rets

,close
2017-01-03 14:30:00,226.580002
2017-01-04 14:30:00,226.399994
2017-01-05 14:30:00,227.210007
2017-01-06 14:30:00,226.460007
2017-01-09 14:30:00,226.460007
2017-01-10 14:30:00,227.100006
2017-01-11 14:30:00,226.529999
2017-01-12 14:30:00,227.050003
2017-01-13 14:30:00,226.250000
2017-01-17 14:30:00,226.750000


In [72]:
rets.iloc[-1]['close'] = ts.tail(1)['open']

In [79]:
rets = rets.shift(1) / rets - 1

In [80]:
rets = rets.dropna()

In [81]:
rets

,close
2017-01-04 14:30:00,0.000795
2017-01-05 14:30:00,-0.003565
2017-01-06 14:30:00,0.003312
2017-01-09 14:30:00,0.000000
2017-01-10 14:30:00,-0.002818
2017-01-11 14:30:00,0.002516
2017-01-12 14:30:00,-0.002290
2017-01-13 14:30:00,0.003536
2017-01-17 14:30:00,-0.002205
2017-01-18 14:30:00,0.003718


In [84]:
rets.columns = [ts.name]

In [85]:
rets

,SPY
2017-01-04 14:30:00,0.000795
2017-01-05 14:30:00,-0.003565
2017-01-06 14:30:00,0.003312
2017-01-09 14:30:00,0.000000
2017-01-10 14:30:00,-0.002818
2017-01-11 14:30:00,0.002516
2017-01-12 14:30:00,-0.002290
2017-01-13 14:30:00,0.003536
2017-01-17 14:30:00,-0.002205
2017-01-18 14:30:00,0.003718


In [60]:




rets.index = rets.index.to_datetime()
rets.index = rets.index.tz_localize("UTC")
rets.columns = [symbol]
return rets

In [59]:
ts.shift(-1)

,close,open
2017-01-03 14:30:00,226.580002,225.619995
2017-01-04 14:30:00,226.399994,226.270004
2017-01-05 14:30:00,227.210007,226.529999
2017-01-06 14:30:00,226.460007,226.910004
2017-01-09 14:30:00,226.460007,226.479996
2017-01-10 14:30:00,227.100006,226.360001
2017-01-11 14:30:00,226.529999,226.500000
2017-01-12 14:30:00,227.050003,226.729996
2017-01-13 14:30:00,226.250000,226.309998
2017-01-17 14:30:00,226.750000,226.539993


In [30]:
ts

2017-01-03 14:30:00    225.039993
2017-01-04 14:30:00    225.619995
2017-01-05 14:30:00    226.270004
2017-01-06 14:30:00    226.529999
2017-01-09 14:30:00    226.910004
2017-01-10 14:30:00    226.479996
2017-01-11 14:30:00    226.360001
2017-01-12 14:30:00    226.500000
2017-01-13 14:30:00    226.729996
2017-01-17 14:30:00    226.309998
2017-01-18 14:30:00    226.539993
2017-01-19 14:30:00    226.839996
2017-01-20 14:30:00    226.699997
2017-01-23 14:30:00    226.740005
2017-01-24 14:30:00    226.399994
2017-01-25 14:30:00    228.699997
2017-01-26 14:30:00    229.399994
2017-01-27 14:30:00    229.419998
2017-01-30 14:30:00    228.169998
2017-01-31 14:30:00    226.979996
2017-02-01 14:30:00    227.529999
2017-02-02 14:30:00    227.619995
2017-02-03 14:30:00    228.820007
2017-02-06 14:30:00    228.869995
2017-02-07 14:30:00    229.380005
2017-02-08 14:30:00    228.940002
2017-02-09 14:30:00    229.240005
2017-02-10 14:30:00    231.000000
2017-02-13 14:30:00    232.080002
2017-02-14 14:

In [35]:
symbol = rtjson['chart']['result'][0]['meta']['symbol']
timestamps = rtjson['chart']['result'][0]['timestamp']
open_price = rtjson['chart']['result'][0]['indicators']['quote'][0]['open']
close_price = rtjson['chart']['result'][0]['indicators']['quote'][0]['close']

In [128]:
d = pd.Series({'open' : pd.Series(open_price, index=pd.to_datetime(timestamps, unit='s')),
     'close' : pd.Series(close_price, pd.to_datetime(timestamps, unit='s'))})

In [130]:
d.close

2017-01-03 14:30:00    225.240005
2017-01-04 14:30:00    226.580002
2017-01-05 14:30:00    226.399994
2017-01-06 14:30:00    227.210007
2017-01-09 14:30:00    226.460007
2017-01-10 14:30:00    226.460007
2017-01-11 14:30:00    227.100006
2017-01-12 14:30:00    226.529999
2017-01-13 14:30:00    227.050003
2017-01-17 14:30:00    226.250000
2017-01-18 14:30:00    226.750000
2017-01-19 14:30:00    225.910004
2017-01-20 14:30:00    226.740005
2017-01-23 14:30:00    226.149994
2017-01-24 14:30:00    227.600006
2017-01-25 14:30:00    229.570007
2017-01-26 14:30:00    229.330002
2017-01-27 14:30:00    228.970001
2017-01-30 14:30:00    227.550003
2017-01-31 14:30:00    227.529999
2017-02-01 14:30:00    227.619995
2017-02-02 14:30:00    227.770004
2017-02-03 14:30:00    229.339996
2017-02-06 14:30:00    228.929993
2017-02-07 14:30:00    228.940002
2017-02-08 14:30:00    229.240005
2017-02-09 14:30:00    230.600006
2017-02-10 14:30:00    231.509995
2017-02-13 14:30:00    232.770004
2017-02-14 14:

In [25]:
pd.to_datetime(ts.index[1])

Timestamp('2017-01-04 14:30:00')

In [26]:
pd.to_datetime(ts.index)

DatetimeIndex(['2017-01-03 14:30:00', '2017-01-04 14:30:00',
               '2017-01-05 14:30:00', '2017-01-06 14:30:00',
               '2017-01-09 14:30:00', '2017-01-10 14:30:00',
               '2017-01-11 14:30:00', '2017-01-12 14:30:00',
               '2017-01-13 14:30:00', '2017-01-17 14:30:00',
               ...
               '2017-11-16 14:30:00', '2017-11-17 14:30:00',
               '2017-11-20 14:30:00', '2017-11-21 14:30:00',
               '2017-11-22 14:30:00', '2017-11-24 14:30:00',
               '2017-11-27 14:30:00', '2017-11-28 14:30:00',
               '2017-11-29 14:30:00', '2017-11-30 14:30:00'],
              dtype='datetime64[ns]', length=231, freq=None)

In [27]:
[ts.size, ts.name, ts.index, ts.dtype]

[231, 'SPY', DatetimeIndex(['2017-01-03 14:30:00', '2017-01-04 14:30:00',
                '2017-01-05 14:30:00', '2017-01-06 14:30:00',
                '2017-01-09 14:30:00', '2017-01-10 14:30:00',
                '2017-01-11 14:30:00', '2017-01-12 14:30:00',
                '2017-01-13 14:30:00', '2017-01-17 14:30:00',
                ...
                '2017-11-16 14:30:00', '2017-11-17 14:30:00',
                '2017-11-20 14:30:00', '2017-11-21 14:30:00',
                '2017-11-22 14:30:00', '2017-11-24 14:30:00',
                '2017-11-27 14:30:00', '2017-11-28 14:30:00',
                '2017-11-29 14:30:00', '2017-11-30 14:30:00'],
               dtype='datetime64[ns]', length=231, freq=None), dtype('float64')]

In [18]:

dt = rtjson_timestamp[1:10]
p = rtjson_open_price[1:10]

NameError: name 'rtjson_timestamp' is not defined

In [17]:
df = pd.Series(p, index=pd.to_datetime(dt,unit='s'))
df

NameError: name 'p' is not defined

In [44]:
start = '1/1/2017'

In [64]:
pd.to_datetime(start)

Timestamp('2017-01-01 00:00:00')

In [63]:
pd.to_datetime(start).strftime('%s')

'1483228800'

In [62]:
datetime.datetime.utcnow()

datetime.datetime(2018, 4, 13, 20, 29, 48, 445452)

In [61]:
symbol = rtjson['chart']['result'][0]['meta']['symbol']
symbol

'SPY'

In [59]:
rtjson_timestamp = rtjson['chart']['result'][0]['timestamp']
rtjson_timestamp

[1483453800,
 1483540200,
 1483626600,
 1483713000,
 1483972200,
 1484058600,
 1484145000,
 1484231400,
 1484317800,
 1484663400,
 1484749800,
 1484836200,
 1484922600,
 1485181800,
 1485268200,
 1485354600,
 1485441000,
 1485527400,
 1485786600,
 1485873000,
 1485959400,
 1486045800,
 1486132200,
 1486391400,
 1486477800,
 1486564200,
 1486650600,
 1486737000,
 1486996200,
 1487082600,
 1487169000,
 1487255400,
 1487341800,
 1487687400,
 1487773800,
 1487860200,
 1487946600,
 1488205800,
 1488292200,
 1488378600,
 1488465000,
 1488551400,
 1488810600,
 1488897000,
 1488983400,
 1489069800,
 1489156200,
 1489411800,
 1489498200,
 1489584600,
 1489671000,
 1489757400,
 1490016600,
 1490103000,
 1490189400,
 1490275800,
 1490362200,
 1490621400,
 1490707800,
 1490794200,
 1490880600,
 1490967000,
 1491226200,
 1491312600,
 1491399000,
 1491485400,
 1491571800,
 1491831000,
 1491917400,
 1492003800,
 1492090200,
 1492435800,
 1492522200,
 1492608600,
 1492695000,
 1492781400,
 1493040600,

In [60]:
rtjson_open_price = rtjson['chart']['result'][0]['indicators']['quote'][0]['open']
rtjson_open_price[1:10]

[225.6199951171875,
 226.27000427246094,
 226.52999877929688,
 226.91000366210938,
 226.47999572753906,
 226.36000061035156,
 226.5,
 226.72999572753906,
 226.30999755859375]